In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Preparing the data

In [ ]:
import pandas as pd

# General list for the whole corpus
corpus_tweets = []
#Syrian dialect
# pandas.read_table for readding .txt files
S_data = pd.read_table('/content/drive/My Drive/Colab Notebooks/NLP - Shami dialect Language Identification /psyrian.txt', names=['Text data'])
# # add labels to the data
S_data['Label'] = 0
print(len(S_data))
# reducing the size of the Dataframe (take the first 1000 index)
S_data = S_data.iloc[:10001,:]
# make a list of tuple for the vectorizer
for tweet in S_data["Text data"]:
  corpus_tweets.append((tweet, 0))

#Lebanees dialect
L_data = pd.read_table('/content/drive/My Drive/Colab Notebooks/NLP - Shami dialect Language Identification /plebanees.txt', names=['Text data'])
L_data['Label'] = 1
print(len(L_data))
# L_data = L_data.iloc[:5001,:]
for tweet in L_data["Text data"]:
  corpus_tweets.append((tweet, 1))
#Palestinian dialect
P_data = pd.read_table('/content/drive/My Drive/Colab Notebooks/NLP - Shami dialect Language Identification /ppalestinian.txt', names=['Text data'])
P_data['Label'] = 2
print(len(P_data))
# P_data = P_data.iloc[:5001,:]
for tweet in P_data["Text data"]:
  corpus_tweets.append((tweet, 2))

#jordinian dialect
J_data = pd.read_table('/content/drive/My Drive/Colab Notebooks/NLP - Shami dialect Language Identification /pjordinian.txt', names=['Text data'])
J_data['Label'] = 3
print(len(J_data))
# J_data = J_data.iloc[:5001,:]
for tweet in J_data["Text data"]:
  corpus_tweets.append((tweet, 3))

37759
10829
10642
7017


# Splitting and encoding the data using: 

TF_IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import random
# split the data for train and test

labels = []
tweets = []
shuffled_corpus_tweets = random.sample(corpus_tweets, len(corpus_tweets))

for item in shuffled_corpus_tweets:
  tweets.append(item[0])
  labels.append(item[1])

x_train, x_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.25, random_state=42) 
x_train_, x_val, y_train_, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42) 

# making the vectorizer
shami_vectorizer = TfidfVectorizer()
shami_vectorizer.fit(x_train)
x_train_encoded = shami_vectorizer.transform(x_train)
x_test_encoded = shami_vectorizer.transform(x_test)

Word Emmbeding

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train)

In [ ]:
x_train_tok = tokenizer.texts_to_sequences(x_train)
x_test_tok = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [ ]:
# Padding the sequence
from keras.preprocessing.sequence import pad_sequences

maxlen = 100
x_train_tok = pad_sequences(x_train_tok, padding='post', maxlen=maxlen)
x_test_tok = pad_sequences(x_test_tok, padding='post', maxlen=maxlen)

# Baseline models

Naive Bayes - GaussianNB -- TFIDF

In [ ]:
# training a Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

GaussianNB_model = GaussianNB().fit(x_train_encoded.toarray() ,y_train)
GaussianNB_predictions = GaussianNB_model.predict(x_test_encoded.toarray())



dict_ = { 0: "Syrian",
          1 : "Lebaneese",
          2 : "Palestinian",
          3 : "Jordanian"}

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

GaussianNB_model_accuracy = GaussianNB_model.score(x_test_encoded.toarray() , y_test)
print(f"Base line model accuracy GaussianNB model TFIDF : {GaussianNB_model_accuracy}")
# print(f"f1_score: {f1_score(y_test, GaussianNB_predictions, average='macro')}")
# print(f"precision_score: {precision_score(y_test, GaussianNB_predictions, average='macro')}")
# print(f"recall_score: {recall_score(y_test, GaussianNB_predictions, average='macro')}") 

print(classification_report(y_test, GaussianNB_predictions))    
for item in range(10):
  index = GaussianNB_predictions[item]
  print(f"Text : {x_test[item]}, \nPredicted_Label : {dict_[index]}, \nReal_label : {y_test[item]}")

Base line model accuracy GaussianNB model TFIDF : 0.668502545983581
              precision    recall  f1-score   support

           0       0.63      0.80      0.70      2554
           1       0.79      0.63      0.70      2652
           2       0.68      0.62      0.65      2605
           3       0.58      0.61      0.60      1812

    accuracy                           0.67      9623
   macro avg       0.67      0.66      0.66      9623
weighted avg       0.68      0.67      0.67      9623

Text :  هاد فرح عم يمر من جنب المشفى حبيبي ماني سامعك بحكي معك من تروح الضجه, 
Predicted_Label : Syrian, 
Real_label : 0
Text : مصاري اناكلت عليك بترجعلك ضعف , 
Predicted_Label : Syrian, 
Real_label : 3
Text :  عهد ابو خشبه ما يتحاسب , 
Predicted_Label : Lebaneese, 
Real_label : 1
Text : و اختها عالسريع ردت اله يعينك فيما ابتلاك يخوي ي حبيبي ي غالي , 
Predicted_Label : Palestinian, 
Real_label : 3
Text :  ماشا اله ماعاد فيه اطفال خلاص , 
Predicted_Label : Lebaneese, 
Real_label : 1
Text :  شك

Naive Bayes - GaussianNB -- Word Embedding

In [ ]:
GaussianNB_model = GaussianNB().fit(x_train_tok, y_train)
GaussianNB_predictions = GaussianNB_model.predict(x_test_tok)

In [ ]:
# GaussianNB_model_accuracy = GaussianNB_model.score(x_test_tok , y_test)
# print(f"Base line model accuracy : {GaussianNB_model_accuracy}")
# print(f"f1_score: {f1_score(y_test, GaussianNB_predictions, average='macro')}")
# print(f"precision_score: {precision_score(y_test, GaussianNB_predictions, average='macro')}")
# print(f"recall_score: {recall_score(y_test, GaussianNB_predictions, average='macro')}")
# print(precision_recall_fscore_support(y_test, GaussianNB_predictions))  
from sklearn.metrics import classification_report

GaussianNB_model_accuracy_word_embedding = GaussianNB_model.score(x_test_tok , y_test)
print(f"Base line model accuracy GaussianNB model word embedding : {GaussianNB_model_accuracy_word_embedding}")
print(classification_report(y_test, GaussianNB_predictions)) 

Base line model accuracy GaussianNB model word embedding : 0.2989712147978801
              precision    recall  f1-score   support

           0       0.31      0.00      0.01      2554
           1       0.29      1.00      0.45      2652
           2       0.39      0.06      0.10      2605
           3       0.34      0.04      0.07      1812

    accuracy                           0.30      9623
   macro avg       0.33      0.27      0.16      9623
weighted avg       0.33      0.30      0.17      9623



OneVsOneClassifier -- TFIDF

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

OneVsOneClassifier_model = OneVsOneClassifier(LinearSVC(random_state=1))
OneVsOneClassifier_model.fit(x_train_encoded.toarray() ,y_train)
OneVsOneClassifier_model_predictions = OneVsOneClassifier_model.predict(x_test_encoded.toarray())

In [ ]:
from sklearn.metrics import classification_report

OneVsOneClassifier_model_accuracy = OneVsOneClassifier_model.score(x_test_encoded.toarray() , y_test)
print(f"Base line model accuracy : {OneVsOneClassifier_model_accuracy}")
# print(f"f1_score: {f1_score(y_test, OneVsOneClassifier_model_predictions, average='macro')}")
# print(f"precision_score: {precision_score(y_test, OneVsOneClassifier_model_predictions, average='macro')}")
# print(f"recall_score: {recall_score(y_test, OneVsOneClassifier_model_predictions, average='macro')}")   

print(classification_report(y_test, OneVsOneClassifier_model_predictions))
for item in range(10):
  index = OneVsOneClassifier_model_predictions[item]
  print(f"Text : {x_test[item]}, \nPredicted_Label : {dict_[index]}, \nReal_label : {y_test[item]}")

Base line model accuracy : 0.8147147459212304
              precision    recall  f1-score   support

           0       0.88      0.87      0.87      2554
           1       0.80      0.86      0.83      2652
           2       0.80      0.80      0.80      2605
           3       0.77      0.69      0.73      1812

    accuracy                           0.81      9623
   macro avg       0.81      0.81      0.81      9623
weighted avg       0.81      0.81      0.81      9623

Text :  هاد فرح عم يمر من جنب المشفى حبيبي ماني سامعك بحكي معك من تروح الضجه, 
Predicted_Label : Syrian, 
Real_label : 0
Text : مصاري اناكلت عليك بترجعلك ضعف , 
Predicted_Label : Jordanian, 
Real_label : 3
Text :  عهد ابو خشبه ما يتحاسب , 
Predicted_Label : Lebaneese, 
Real_label : 1
Text : و اختها عالسريع ردت اله يعينك فيما ابتلاك يخوي ي حبيبي ي غالي , 
Predicted_Label : Jordanian, 
Real_label : 3
Text :  ماشا اله ماعاد فيه اطفال خلاص , 
Predicted_Label : Palestinian, 
Real_label : 1
Text :  شكلك حساسه م تدى مشى 

OneVsOneClassifier -- Word Embedding

In [ ]:
OneVsOneClassifier_model = OneVsOneClassifier(LinearSVC(random_state=1))
OneVsOneClassifier_model.fit(x_train_tok, y_train)
OneVsOneClassifier_model_predictions = OneVsOneClassifier_model.predict(x_test_tok)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

In [ ]:
from sklearn.metrics import classification_report

OneVsOneClassifier_model_accuracy = OneVsOneClassifier_model.score(x_test_tok , y_test)
print(f"Base line model accuracy : {OneVsOneClassifier_model_accuracy}")
OneVsOneClassifier_model_accuracy = OneVsOneClassifier_model.score(x_test_tok , y_test)
# classification report
print(classification_report(y_test, OneVsOneClassifier_model_predictions))

Base line model accuracy : 0.2840070664034085
              precision    recall  f1-score   support

           0       0.29      0.04      0.07      2554
           1       0.32      0.64      0.42      2652
           2       0.29      0.16      0.21      2605
           3       0.21      0.29      0.24      1812

    accuracy                           0.28      9623
   macro avg       0.28      0.28      0.24      9623
weighted avg       0.28      0.28      0.24      9623



# Deep Learning

Building the model

In [ ]:
import keras
from keras.models import Sequential
from keras import layers
from keras.layers import LSTM, Dense, GRU, Embedding, Bidirectional, Dropout

Embedding_dim = 100
#23683
model = Sequential()
# model.add(layers.Embedding(vocab_size, Embedding_dim, input_length=maxlen))
# InputLayer( input_shape=(img_width, img_heigh, img_ch)) #input_tensor=img1,
# model.add(layers.Embedding(len(shami_vectorizer.get_feature_names()), Embedding_dim, input_length=24010)) #tfidf
model.add(layers.Embedding(vocab_size, Embedding_dim, input_length=maxlen)) # Embedding
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(len(x_train[0]))))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='softmax'))


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          6590600   
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 54)                27648     
_________________________________________________________________
dense (Dense)                (None, 10)                550       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
Total params: 6,618,853
Trainable params: 6,618,853
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

filepath = "/content/drive/MyDrive/Colab Notebooks/NLP - Shami dialect Language Identification /LanguageModel.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]
history = model.fit(x_train_encoded.toarray(), np.asarray(y_train),
                    epochs=3,
                    verbose=True,
                    validation_data=(x_test_encoded.toarray(), np.asarray(y_test)),
                    batch_size=10,
                    callbacks=desired_callbacks)
train_loss, train_accuracy = model.evaluate(x_train_encoded.toarray(), np.asarray(y_train).astype(np.float32), verbose=False)
print("Training Accuracy: {:.4f}".format(train_accuracy))
test_loss, test_accuracy = model.evaluate(x_test_encoded.toarray(), np.asarray(y_test).astype(np.float32), verbose=False)
print("Testing Accuracy:  {:.4f}".format(test_accuracy))

Epoch 1/3
   3/2887 [..............................] - ETA: 307:43:39 - loss: 1.6095 - accuracy: 0.2778

Tokenize for the train - validation - test.

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(x_train_)

In [ ]:
x_train_tok = tokenizer.texts_to_sequences(x_train_)
x_val = tokenizer.texts_to_sequences(x_val)
x_test_tok = tokenizer.texts_to_sequences(x_test)

In [ ]:
# Padding the sequence
from keras.preprocessing.sequence import pad_sequences

maxlen = 100
x_train_tok_ = pad_sequences(x_train_tok, padding='post', maxlen=maxlen)
x_val_tok_ = pad_sequences(x_val, padding='post', maxlen=maxlen)
x_test_tok = pad_sequences(x_test_tok, padding='post', maxlen=maxlen)

In [ ]:
# import keras
# from keras.models import Sequential
from keras import layers
from keras.layers import LSTM, Dense, GRU, Embedding, Bidirectional, Dropout
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization,LeakyReLU
from tensorflow.python.keras import regularizers
Embedding_dim = 100
#23683
filters = 7
kernel_size = (7,)
lstm_units = 100
# word_index = tokenizer.word_index
# text_embedding = np.zeros((len(word_index) + 1, 300))

model = Sequential()
# model.add(Embedding(input_dim=text_embedding.shape[0],output_dim=text_embedding.shape[1],
                    # weights=[text_embedding],input_length=maxlen,trainable=True))
model.add(layers.Embedding(vocab_size, Embedding_dim, input_length=maxlen))
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters, kernel_size=kernel_size,kernel_regularizer=regularizers.l2(0.00001), padding='same'))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(lstm_units,dropout=0.5, recurrent_dropout=0.5,return_sequences=True)))
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters, kernel_size=kernel_size,kernel_regularizer=regularizers.l2(0.00001), padding='same'))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(lstm_units,dropout=0.5, recurrent_dropout=0.5,return_sequences=True)))
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters, kernel_size=kernel_size,kernel_regularizer=regularizers.l2(0.00001), padding='same'))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(lstm_units,dropout=0.5, recurrent_dropout=0.5)))
model.add(Dense(4,activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

filepath = "/content/drive/MyDrive/Colab Notebooks/NLP - Shami dialect Language Identification /LanguageModel.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]
history = model.fit(x_train_tok_, np.asarray(y_train_),
                    epochs=6,
                    verbose=True,
                    validation_data=(x_val_tok_, np.asarray(y_val)),
                    batch_size=10,
                    callbacks=desired_callbacks)
train_loss, train_accuracy = model.evaluate(x_train_tok_, np.asarray(y_train_).astype(np.float32), verbose=False)
print("Training Accuracy: {:.4f}".format(train_accuracy))
test_loss, test_accuracy = model.evaluate(x_test_tok, np.asarray(y_test).astype(np.float32), verbose=False)
print("Testing Accuracy:  {:.4f}".format(test_accuracy))

Epoch 1/6
2165/2165 [==============================] - 335s 149ms/step - loss: 1.2794 - accuracy: 0.3465 - val_loss: 0.9493 - val_accuracy: 0.6044

Epoch 00001: loss improved from inf to 1.16261, saving model to /content/drive/MyDrive/Colab Notebooks/NLP - Shami dialect Language Identification /LanguageModel.hdf5
Epoch 2/6
2165/2165 [==============================] - 344s 159ms/step - loss: 0.8849 - accuracy: 0.6376 - val_loss: 0.7250 - val_accuracy: 0.7245

Epoch 00002: loss improved from 1.16261 to 0.84517, saving model to /content/drive/MyDrive/Colab Notebooks/NLP - Shami dialect Language Identification /LanguageModel.hdf5
Epoch 3/6
2165/2165 [==============================] - 370s 171ms/step - loss: 0.6580 - accuracy: 0.7596 - val_loss: 0.6735 - val_accuracy: 0.7560

Epoch 00003: loss improved from 0.84517 to 0.65526, saving model to /content/drive/MyDrive/Colab Notebooks/NLP - Shami dialect Language Identification /LanguageModel.hdf5
Epoch 4/6
2165/2165 [==========================

In [ ]:
from keras.models import load_model
from sklearn.metrics import classification_report, precision_recall_fscore_support
import numpy as np

emb_model = load_model("/content/drive/MyDrive/Colab Notebooks/NLP - Shami dialect Language Identification /LanguageModel.hdf5")
filepath = "/content/drive/MyDrive/Colab Notebooks/NLP - Shami dialect Language Identification /LanguageModel.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]
history = emb_model.fit(x_train_tok_, np.asarray(y_train_),
                    epochs=3,
                    verbose=True,
                    validation_data=(x_val_tok_, np.asarray(y_val)),
                    batch_size=10,
                    callbacks=desired_callbacks)
train_loss, train_accuracy = emb_model.evaluate(x_train_tok_, np.asarray(y_train_).astype(np.float32), verbose=False)
print("Training Accuracy: {:.4f}".format(train_accuracy))
test_loss, test_accuracy = emb_model.evaluate(x_test_tok, np.asarray(y_test).astype(np.float32), verbose=False)
print("Testing Accuracy:  {:.4f}".format(test_accuracy))
y_pred1 = emb_model.predict(x_test_tok)
pred = []
for item in y_pred1:
  pred.append(np.argmax(item))

print(classification_report(y_test, pred))
print(precision_recall_fscore_support(y_test, pred))   

Epoch 1/3
2165/2165 [==============================] - 338s 152ms/step - loss: 0.4238 - accuracy: 0.8568 - val_loss: 0.7028 - val_accuracy: 0.7811

Epoch 00001: loss improved from inf to 0.42378, saving model to /content/drive/MyDrive/Colab Notebooks/NLP - Shami dialect Language Identification /LanguageModel.hdf5
Epoch 2/3
2165/2165 [==============================] - 334s 154ms/step - loss: 0.4106 - accuracy: 0.8630 - val_loss: 0.6785 - val_accuracy: 0.7865

Epoch 00002: loss improved from 0.42378 to 0.41057, saving model to /content/drive/MyDrive/Colab Notebooks/NLP - Shami dialect Language Identification /LanguageModel.hdf5
Epoch 3/3
2165/2165 [==============================] - 330s 152ms/step - loss: 0.3785 - accuracy: 0.8735 - val_loss: 0.7084 - val_accuracy: 0.7829

Epoch 00003: loss improved from 0.41057 to 0.37847, saving model to /content/drive/MyDrive/Colab Notebooks/NLP - Shami dialect Language Identification /LanguageModel.hdf5
Training Accuracy: 0.9173
Testing Accuracy:  0.